<a href="https://colab.research.google.com/github/erinmcmahon26/Chatbot/blob/main/first_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Basic Chatbot Set Up with ChatterBot

In [ ]:
!pip install chatterbot
!pip install chatterbot_corpus

     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 1.3 MB 7.9 MB/s 
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.36
    Uninstalling SQLAlchemy-1.4.36:
      Successfully uninstalled SQLAlchemy-1.4.36
     |████████████████████████████████| 117 kB 4.1 MB/s 


In [ ]:
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer

In [ ]:
my_bot = ChatBot(name="ZeldaBot", read_only = True, 
                 logic_adapters = ['chatterbot.logic.MathematicalEvaluation', 'chatterbot.logic.BestMatch'])

In [ ]:
small_talk = ['hi, there!',
              'what\'s up,',
              'fine, you?',
              'i really like Zelda videogames',
              'i don\'t understand',
              'sorry, can you repeat?',
              'what is your name?',
              'hello!',
              'do you link videogames?',
              'zelda: breath of the wild is a great video game',
              'how are you doing?',
              'i\'m zelda_bot, as me a question about zelda']
zelda_talk_1 = ['Zelda', 'Link is the main character.']
zelda_talk_2 = ['Link', 'The game is set 100 years after the Calamity.']

In [ ]:
list_trainer = ListTrainer(my_bot)
for item in (small_talk, zelda_talk_1, zelda_talk_2):
  list_trainer.train(item)

List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%


In [ ]:
print(my_bot.get_response("hi"))

sorry, can you repeat?


In [ ]:
print(my_bot.get_response("who is link?"))

hello!


Obviously, not doing great so far.

In [ ]:
from chatterbot.trainers import ChatterBotCorpusTrainer

In [ ]:
corpus_trainer = ChatterBotCorpusTrainer(my_bot)
corpus_trainer.train('chatterbot.corpus.english')

Training ai.yml: [####################] 100%
Training botprofile.yml: [####################] 100%
Training computers.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training emotion.yml: [####################] 100%
Training food.yml: [####################] 100%
Training gossip.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training health.yml: [####################] 100%
Training history.yml: [####################] 100%
Training humor.yml: [####################] 100%
Training literature.yml: [####################] 100%
Training money.yml: [####################] 100%
Training movies.yml: [####################] 100%
Training politics.yml: [####################] 100%
Training psychology.yml: [####################] 100%
Training science.yml: [####################] 100%
Training sports.yml: [####################] 100%
Training trivia.yml: [####################] 100%


In [ ]:
print(my_bot.get_response("hi"))

How are you doing?


In [ ]:
print(my_bot.get_response("I have a swollen foot, what should I do?"))

I am very interested in history, too. what period do you like?


## Chatbot with Zelda Data

In [41]:
import pandas as pd
import numpy as np
import json
import random
import re
import string

# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# import nltk
# from nltk.stem import WordNetLemmatizer
# from nltk.corpus import stopwords

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential 
from keras.layers import Dense, Activation, Dropout

from keras.layers import Dense, Embedding, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

https://colab.research.google.com/notebooks/io.ipynb

https://towardsdatascience.com/how-to-build-your-own-chatbot-using-deep-learning-bb41f970e281

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [36]:
with open('/content/drive/My Drive/intents.json') as file:
    data = json.load(file)

In [37]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [38]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [39]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [42]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [43]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 28)                476       
                                                                 
Total params: 17,020
Trainable params: 17,020
Non-trainable params: 0
__________________________________________________

In [44]:
epochs = 550
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/550
4/4 [==============================] - 3s 5ms/step - loss: 3.3329 - accuracy: 0.0495
Epoch 2/550
4/4 [==============================] - 0s 4ms/step - loss: 3.3317 - accuracy: 0.0495
Epoch 3/550
4/4 [==============================] - 0s 4ms/step - loss: 3.3309 - accuracy: 0.0495
Epoch 4/550
4/4 [==============================] - 0s 4ms/step - loss: 3.3305 - accuracy: 0.0495
Epoch 5/550
4/4 [==============================] - 0s 4ms/step - loss: 3.3300 - accuracy: 0.0396
Epoch 6/550
4/4 [==============================] - 0s 4ms/step - loss: 3.3295 - accuracy: 0.0594
Epoch 7/550
4/4 [==============================] - 0s 4ms/step - loss: 3.3291 - accuracy: 0.0495
Epoch 8/550
4/4 [==============================] - 0s 5ms/step - loss: 3.3287 - accuracy: 0.0495
Epoch 9/550
4/4 [==============================] - 0s 4ms/step - loss: 3.3281 - accuracy: 0.0495
Epoch 10/550
4/4 [==============================] - 0s 4ms/step - loss: 3.3275 - accuracy: 0.0495
Epoch 11/550
4/4 [===========

In [45]:
# saving model
model.save("chat_model")

import pickle

# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

# saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model/assets


In [46]:
!pip3 install colorama

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [48]:
import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "Question: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "end":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type end to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
Question: What is BOTW?
ChatBot: Zelda: Breath of the Wild is a video game produced by Nintendo. An entry in the longrunning The Legend of Zelda series, it was released for the Nintendo Switch and Wii U consoles on March 3, 2017. Breath of the Wild is set at the end of the series' timeline; the player controls Link, who awakens from a hundred-year slumber to defeat Calamity Ganon before it can destroy the kingdom of Hyrule. Similarly to the original Legend of Zelda (1986), players are given little instruction and can explore the open world freely. Tasks include collecting multipurpose items to aid in objectives or solving puzzles and side quests for rewards. The world is unstructured and designed to reward experimentation, and the story can be completed in a nonlinear fashion.
Question: Who is link?
ChatBot: Link is the centre of everything in Nintendos The Legend of Zelda: Breath of the Wild. Hes who you play as in the game, hes the on